In [1]:
import os
import pandas as pd
from cuppa.sample_data.cuppa_features import FeatureLoaderNew
from cuppa.classifier.cuppa_classifier import CuppaClassifier

In [2]:
## Suppress log messages
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [3]:
## Set working directory as the path of the pycuppa package
os.chdir("../..")

## Load features

The features tsv file is formatted as a long form dataframe.

In [4]:
features_path = "cuppa/resources/mock_data/input_data/new_format/prostate_sample.cuppa_data.tsv.gz"
pd.read_table(features_path)

,Source,Category,Key,Value
DNA,SNV96,C>A_ACA,131.0,NaN
DNA,SNV96,C>A_ACC,85.0,NaN
DNA,SNV96,C>A_ACG,19.0,NaN
DNA,SNV96,C>A_ACT,87.0,NaN
DNA,SNV96,C>A_CCA,115.0,NaN
...,...,...,...,...
RNA,ALT_SJ,17;80059742;80061789,1.0,NaN
RNA,ALT_SJ,17;80059742;80064051,1.0,NaN
RNA,ALT_SJ,17;80109649;80110400,3.0,NaN
RNA,ALT_SJ,17;80841503;80842021,1.0,NaN


However, this tsv file needs to be parsed into a pandas dataframe of shape n_samples x n_features.
Note that this example data only has one sample.

In [5]:
loader = FeatureLoaderNew(path=features_path, verbose=True)
features = loader.load()
features

,snv96.C>A_ACA,snv96.C>A_ACC,snv96.C>A_ACG,snv96.C>A_ACT,snv96.C>A_CCA,...,alt_sj.17;80059742;80061789,alt_sj.17;80059742;80064051,alt_sj.17;80109649;80110400,alt_sj.17;80841503;80842021,alt_sj.17;80841716;80842021
sample_1,131.0,85.0,19.0,87.0,115.0,...,1.0,1.0,3.0,1.0,2.0


<br>
The prefix of the feature names denote the feature type. E.g. the feature "snv96.C>A_ACA" has the feature type "snv96". Below are the names of all feature types.

In [6]:
list(features.feat_types)

['snv96', 'event', 'gen_pos', 'sig', 'gene_exp', 'alt_sj']

## Load classifier

A pre-trained CUPPA classifier is stored in the resources directory. Below is an sklearn diagram of the structure of the classifer.

In [7]:
classifier = CuppaClassifier.from_file(path="cuppa/resources/cuppa_classifier.pickle.gz")
classifier

CuppaClassifier(steps=[('sub_clfs', ColumnTransformer(n_jobs=5,
                  transformers=[('gen_pos',
                                 Pipeline(steps=[('add_noise',
                                                  NoiseProfileAdder(agg_func='sum',
                                                                    count_ceiling=10000,
                                                                    noise_counts=500)),
                                                 ('cos_sim',
                                                  ProfileSimilarityTransformer(count_ceiling=10000,
                                                                               feature_prefix='gen_pos.')),
                                                 ('non_best_scale',
                                                  NonBestSimilarityScaler(exponent=5)),
                                                 ('logistic_regression',
                                                  LogisticRegression(C=1,
                                                                     class_we...
                                                  ProfileSimilarityTransformer(agg_func='mean',
                                                                               feature_prefix='alt_sj.')),
                                                 ('non_best_scaler',
                                                  NonBestSimilarityScaler(exponent=2)),
                                                 ('logistic_regression',
                                                  LogisticRegression(C=4,
                                                                     class_weight='balanced',
                                                                     max_iter=10000,
                                                                     multi_class='multinomial',
                                                                     penalty='l1',
                                                                     random_state=0,
                                                                     solver='saga'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x2b35753d0>)],
                  verbose=True)), ('meta_clfs', ColumnTransformer(n_jobs=5,
                  transformers=[('dna_combined',
                                 Pipeline(steps=[('logistic_regression',
                                                  LogisticRegression(C=1,
                                                                     class_weight='balanced',
                                                                     max_iter=1000,
                                                                     multi_class='multinomial',
                                                                     random_state=0,
                                                                     solver='saga')),
                                                 ('calibrator',
                                                  RollingAvgCalibration(min_true_samples=10,
                                                                        window_size='variable')),
                                                 ('fusion_overrider',
                                                  FusionProbOverrider(_is_fitted=True...
                                                              use_first_col=True)),
                                                 ('logistic_regression',
                                                  LogisticRegression(C=1,
                                                                     class_weight='balanced',
                                                                     max_iter=1000,
                                                                     multi_class='multinomial',
                                                                     random_state=0,
                                       

## Raw predictions

To save disk space, features that are absent in a sample are excluded from the features tsv file. 

We can see that our sample has fewer features than those required by the classifier.

In [8]:
print("No. features in sample: " + str(features.shape[1]))

No. features in sample: 49813


In [9]:
print("No. features in classifier: " + str(len(classifier.required_features)))

print("\nNo. features per feature type:")
for feature_type, feature_names in classifier.required_features_by_type.items():
    print("  " + feature_type + ": " + str(len(feature_names)) )

No. features in classifier: 79252

No. features per feature type:
  gen_pos: 6101
  snv96: 96
  event: 335
  sig: 5
  gene_exp: 5893
  alt_sj: 66822


<br>We therefore need to fill in the missing features that are stored in the classifier.

In [10]:
features_filled = classifier.fill_missing_cols(features)

<br>Then we can make the predictions.

In [11]:
prediction = classifier.predict(features_filled)

<br>The prediction output is a dataframe with multi-indexed rows containing various types of information. The columns are the cancer types.

In [12]:
prediction.iloc[:,0:5]

Anogenital  Bone/Soft tissue: Cartilaginous neoplasm  Bone/Soft tissue: GIST  Bone/Soft tissue: Leiomyosarcoma  Bone/Soft tissue: Liposarcoma
sample_id data_type      clf_group clf_name     feat_name   feat_value                                                                                                                                               
sample_1  prob           combined  combined     NaN         NaN           0.000100                                       NaN                0.000100                          0.000100                       0.000100
                         dna       dna_combined NaN         NaN           0.000000                              1.481489e-05                0.000000                          0.000000                       0.000000
                         rna       rna_combined NaN         NaN           0.000000                                       NaN                0.000000                          0.000000                       0.000972
                         dna       gen_pos      NaN         NaN           0.000007                              7.943869e-08                0.000031                          0.000006                       0.000003
                                   snv96        NaN         NaN           0.026057                              2.691297e-06                0.000300                          0.002645                       0.001093
...                                                                            ...                                       ...                     ...                               ...                            ...
NaN       cv_performance rna       alt_sj       n_correct   NaN          29.000000                              0.000000e+00               27.000000                         17.000000                       3.000000
                                                n_predicted NaN          46.000000                              0.000000e+00               27.000000                         20.000000                       3.000000
                                                n_total     NaN          46.000000                              0.000000e+00               27.000000                         19.000000                       5.000000
                                                precision   NaN           0.630435                                       NaN                1.000000                          0.850000                       1.000000
                                                recall      NaN           0.630435                                       NaN                1.000000                          0.894737                       0.600000

[389 rows x 5 columns]

The prediction output contains the following data types:
* _prob_: cancer type probabilities
* _feat_contrib_: contributions of each feature to each cancer type
* _sig_quantile_: mutational signature quantile relative to the cancer type training cohort
* _cv_performance_: cross-validation performance

In [13]:
prediction.index.get_level_values("data_type").unique().tolist()

['prob', 'feat_contrib', 'sig_quantile', 'cv_performance']

## Prediction interpretation

The above prediction output does not easily allow us to conclude the cancer type of the sample. The summarize() method returns (in this case) the top 2 cancer type predictions per classifier. 

We can see that this sample is confidently predicted as a prostate cancer. We can also see that this sample being male and having a TMPRSS2-ERG fusion contributed to the EVENT classifier prediction.

In [14]:
pred_summ = prediction.summarize(top_n_classes=2, show_extra_info=True)
pred_summ

,sample_id,clf_group,clf_name,pred_class_1,pred_class_2,pred_prob_1,pred_prob_2,extra_info,extra_info_format
0,sample_1,combined,combined,Prostate,Urothelial tract,0.996795,0.000100,NaN,NaN
1,sample_1,dna,dna_combined,Prostate,Bone/Soft tissue: Cartilaginous neoplasm,0.999985,0.000015,NaN,NaN
2,sample_1,rna,rna_combined,Prostate,NET: Lung,0.995301,0.001875,NaN,NaN
3,sample_1,dna,gen_pos,Prostate,Bone/Soft tissue: Other,0.999176,0.000119,NaN,NaN
4,sample_1,dna,snv96,Prostate,HPB: Pancreas,0.643410,0.083093,"AID/APOBEC (SBS2/13)=295.9,4.4%; UV (SBS7)=62....","{sig_name}={count},{percent}"
5,sample_1,dna,event,Prostate,Bone/Soft tissue: Other,0.997120,0.000473,"1: fusion.TMPRSS2_ERG=1.0,6.8; trait.is_male=1...","{pred_class_n}: {feat_name}={value},{log_odds}"
6,sample_1,rna,gene_exp,Prostate,Skin: Other,0.998799,0.000891,NaN,NaN
7,sample_1,rna,alt_sj,Prostate,Head and neck: Other,0.997567,0.001188,NaN,NaN


<br>We can also generate a visualization by calling the plot() method. This will create an image as exampled in the README.

In [3]:
prediction.plot(plot_path="cuppa_vis.png")